In [1]:
import pandas as pd
from glom import glom

c:\Users\p0pp1\miniconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
""" Get stuff out of Netfile v2 API
"""
from pprint import PrettyPrinter
from pathlib import Path
import os
import requests

BASE_URL = 'https://netfile.com/api/campaign'
CONTRIBUTION_FORM = 'F460A'
EXPENDITURE_FORM = 'F460E'

PARAMS = { 'aid': 'COAK' }

def get_auth_from_env_file(filename: str='.env'):
    """ Split .env file on newline and look for API_KEY and API_SECRET
        Return their values as a tuple
    """
    env_file=Path(filename)
    auth_keys = [ 'API_KEY', 'API_SECRET' ]
    if env_file.exists():
        auth = tuple( v for _, v in sorted([
            ln.split('=') for ln in
            env_file.read_text(encoding='utf8').strip().split('\n')
            if ln.startswith(auth_keys[0]) or ln.startswith(auth_keys[1])
        ], key=lambda ln: auth_keys.index(ln[0])))
    else:
        auth=tuple(os.environ[key] for key in auth_keys)
            
    return auth

AUTH=get_auth_from_env_file()

pp = PrettyPrinter()

def get_filing(offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    body = res.json()
    results = body.pop('results')

    return results, body
def get_filer(filer_nid):
    """ Get one filer
    """
    url = f'{BASE_URL}/filer/v101/filers'

    res = requests.get(url, params={ **PARAMS, 'filerNid': filer_nid }, auth=AUTH)
    body = res.json()

    return body['results']

In [3]:
filing=get_filing()

In [4]:
print(filing[1])
print(len(filing[0]))

{'totalCount': 5543, 'totalType': 'Exact', 'empty': False, 'count': 1000, 'hasNextPage': True, 'hasPreviousPage': False, 'limit': 1000, 'offset': 0, 'description': 'Filings', 'querySort': None, 'pageNumber': 1}
1000


In [5]:
file=filing[0][0]
filer_id=file.get('filerMeta',{}).get('filerId', 'None')
filer=get_filer(filer_id)
filer

[{'filerNid': '204680988',
  'visibilityLevel': 'Staff',
  'filerName': 'Oakland Neighbors for Affordable Housing and Safe Street, Yes on U, 2022',
  'candidateName': None,
  'aid': 'COAK',
  'adsid': 163882,
  'isTerminated': False,
  'committeeTypes': ['Primarily Formed Measure'],
  'nameHistory': [],
  'addressList': [{'addressTypes': 'Disclosure, Mailing',
    'line1': '5111 Telegraph Avenue #203',
    'line2': '',
    'city': 'Oakland',
    'state': 'CA',
    'zip': '94609'}],
  'phoneList': [{'phoneType': 'Other', 'number': '5108824536'}],
  'emailList': [{'address': 'ssjreyes@comast.net',
    'emailType': 'Notification'}],
  'statusItemList': [{'status': 'ACTIVE',
    'date': '2022-09-16T21:14:00.1326589+00:00'}],
  'registrations': {'CA SOS': '1452659'},
  'officers': [],
  'electionInfluences': [{'influenceNid': '204680995',
    'filerNid': '204680988',
    'startDate': '2022-06-08',
    'endDate': None,
    'adsid': 163882,
    'electionNid': '178981824',
    'seatNid': None,

In [6]:
files=filing[0]
row=[]
failed=[]
filer_ids=[]
filers={'candidate_name':[],'is_terminated':[],'type':[],'election_year':[],'contest':[],'incumbent':[],'start':[],'end':[],'is_winner':[]}
filings={'form_type':[],'fppc_id':[],'filer_id':[],'filer_name':[],'filing_id':[],'filing_date':[],}
for file in files:
    
    form_type=file.get('specificationRef',{}).get('name', 'None')

    filer_name=file.get('filerMeta',{}).get('commonName', 'None')

    filer_id=file.get('filerMeta',{}).get('filerId', 'None')
    filer_ids.append(filer_id)
    filing_id=file.get('filingMeta',{}).get('issuedFilingId', 'None')

    filing_date=file.get('filingMeta',{}).get('legalFilingDate', 'None')

    fppc_id=file.get('filerMeta',{}).get('strings',{}).get('Registration_CA SOS', 'None')

    filings['form_type'].append(form_type)
    filings['filer_name'].append(filer_name)
    filings['filer_id'].append(filer_id)
    filings['filing_id'].append(filing_id)
    filings['filing_date'].append(filing_date)
    filings['fppc_id'].append(fppc_id)

In [7]:
df=pd.DataFrame(filings)
df

,form_type,fppc_id,filer_id,filer_name,filing_id,filing_date
0,FPPC460,1452659,204680988,Oakland Neighbors for Affordable Housing and S...,209846990,2024-01-29
1,FPPC460,1448165,203581413,Kyra Mungia for Oakland Unified School Board 2022,209807970,2024-01-29
2,FPPC410,1465054,209723026,Nikki F Bas for Supervisor 2024,209728029,2024-01-25
3,FPPC460,1465054,209723026,Nikki F Bas for Supervisor 2024,209724400,2024-01-25
4,FPPC460,1465054,209723026,Nikki F Bas for Supervisor 2024,209724294,2024-01-25
...,...,...,...,...,...,...
995,FPPC410,1442227,201133776,Hodge for Oakland Mayor 2022,201812779,2022-01-26
996,FPPC460,1430558,189811827,Maiya Edgerly for School Board 2020,201796832,2022-01-25
997,FPPC460,1310647,122954602,The Oakland Fund,201755417,2022-01-21
998,FPPC460,1431009,192237541,Families in Action for Justice Fund,201718783,2022-01-18


filer_ids=set(filer_ids)
for filer_id in filer_ids:    
    filer=get_filer(filer_id)

    com_type=filer[0].get('committeeTypes', 'None')

    if com_type == ['Candidate or Officeholder']:
            candidate_name=filer[0].get('candidateName', 'None')
                
            is_terminated=filer[0].get('isTerminated', 'None')
            try:
                filer[0].get('electionInfluences',{})[0]
                electionInfluences=filer[0].get('electionInfluences',{})
                contest=electionInfluences[0].get('seat',{}).get('officeName', 'None')
                incumbent=filer[0].get('electionInfluences',{})[0].get('candidate',{}).get('isIncumbent', 'None')
                    
                start=filer[0].get('electionInfluences',{})[0].get('startDate', 'None')
                    
                end=filer[0].get('electionInfluences',{})[0].get('endDate', 'None')
                    
                is_winner=filer[0].get('electionInfluences',{})[0].get('candidate',{}).get('isWinner', 'None')

                election_year=filer[0].get('electionInfluences',{})[0].get('electionDate')[:4]
            except:
                failed.append(filer)
    filers['candidate_name'].append(candidate_name)
    filers['contest'].append(contest)
    filers['election_year'].append(election_year)
    filers['end'].append(end)
    filers['incumbent'].append(incumbent)
    filers['is_terminated'].append(is_terminated)
    filers['is_winner'].append(is_winner)
    filers['start'].append(start)
    filers['type'].append(com_type)

In [8]:
df2=pd.DataFrame(filers)
df2

,candidate_name,is_terminated,type,election_year,contest,incumbent,start,end,is_winner


In [9]:
def get_elections(id, offset=0):
    """ Get a elections
    """
    url = f'{BASE_URL}/election/v101/elections'

    params = { **PARAMS, 'ElectionNid':id}
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    print(res)
    body = res.json()
    results = body['results']

In [10]:
print(get_elections('208728634'))

<Response [200]>
None


In [11]:
def list_elections():
    """ Get all the elections
    """
    url = f'{BASE_URL}/election/v101/elections'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    body = res.json()

    return body['results']

In [12]:
elections=list_elections()
elections[0]['candidates']


[{'candidateNid': '180842909',
  'candidateName': 'Davis, Benjamin "Sam"',
  'seatNid': '207972561',
  'isIncumbent': False,
  'isWinner': False,
  'partyAffiliationId': '0',
  'partyAffiliationName': ''},
 {'candidateNid': '208539738',
  'candidateName': 'Dunbar, Donnel C',
  'seatNid': '207972552',
  'isIncumbent': False,
  'isWinner': False,
  'partyAffiliationId': '0',
  'partyAffiliationName': ''},
 {'candidateNid': '161700164',
  'candidateName': 'Fife, Carroll',
  'seatNid': '207972552',
  'isIncumbent': False,
  'isWinner': False,
  'partyAffiliationId': '0',
  'partyAffiliationName': ''},
 {'candidateNid': '208555126',
  'candidateName': 'Frank, Edward C',
  'seatNid': '207972521',
  'isIncumbent': False,
  'isWinner': False,
  'partyAffiliationId': '0',
  'partyAffiliationName': ''},
 {'candidateNid': '208539506',
  'candidateName': 'Goolsby, Merika',
  'seatNid': '200879024',
  'isIncumbent': False,
  'isWinner': False,
  'partyAffiliationId': '0',
  'partyAffiliationName': 

In [13]:
i=1
election_list=[]
previous_df=pd.DataFrame()
for election in elections:
    candidates=election['candidates']
    seats=election['seats']
    election_name=election['electionCaption']
    electionNid=election['electionNid']
    election_key={'election_name':election_name, 'electionNid':electionNid}
    election_list.append(election_key)
    if candidates and seats:
        seat_df=pd.DataFrame(seats)
        candidate_df=pd.DataFrame(candidates)
        merge_df=candidate_df.merge(seat_df, on='seatNid')
        current_df=merge_df[['candidateNid','candidateName','seatNid','officeName','electionNid']]
        previous_df=pd.concat([previous_df,current_df],ignore_index=True)
    #merge_df=merge_df.merge(election_key, on='electionNid')
    print(i)
    i+=1
election_df=pd.DataFrame(election_list)
final_df=previous_df.merge(election_df, on='electionNid')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [14]:
final_df.head(15)

,candidateNid,candidateName,seatNid,officeName,electionNid,election_name
0,180842909,"Davis, Benjamin ""Sam""",207972561,Director - Oakland Unified School District - 1,200879019,11/5/2024 - General
1,208539738,"Dunbar, Donnel C",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General
2,161700164,"Fife, Carroll",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General
3,208555126,"Frank, Edward C",207972521,City Council - City of Oakland - 1,200879019,11/5/2024 - General
4,208539506,"Goolsby, Merika",200879024,City Council - City of Oakland - 7,200879019,11/5/2024 - General
5,208555214,"Herndon, Shantell",207972521,City Council - City of Oakland - 1,200879019,11/5/2024 - General
6,208555055,"Hirsch, Shan M",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General
7,158244435,"Jordan, Tyron",207185813,City Council Member At-Large - City of Oakland,200879019,11/5/2024 - General
8,208504223,"Logan, Warren",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General
9,209761019,"Love, Tonya",207185813,City Council Member At-Large - City of Oakland,200879019,11/5/2024 - General
